# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import map_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
csv_file = os.path.join("C://Users/Thissen/Python-API-Challenge/WeatherPy/Output/city_data.csv")

city_df = pd.read_csv(csv_file)
city_df

,Unnamed: 0,City ID,City Name,Country,Lat,Lon,Temp,Humidity,Cloudiness,Wind Speed
0,0,2126123,Chokurdakh,RU,70.6333,147.9167,-34.83,100,95,6.76
1,1,3372964,Lagoa,PT,39.0500,-27.9833,62.71,83,1,7.87
2,2,6201424,Mataura,NZ,-46.1927,168.8643,74.70,36,100,3.91
3,3,3361934,Saldanha,ZA,-33.0117,17.9442,57.25,75,0,17.83
4,4,2264557,Ponta do Sol,PT,32.6667,-17.1000,64.35,72,17,6.20
...,...,...,...,...,...,...,...,...,...,...
583,583,2038067,Chifeng,CN,42.2683,118.9636,32.41,37,91,21.79
584,584,2386756,Gamboula,CF,4.1178,15.1393,70.32,84,88,2.66
585,585,694558,Semypolky,UA,50.7167,30.9333,29.52,75,88,12.59
586,586,2176639,Batemans Bay,AU,-35.7167,150.1833,75.22,59,4,9.22


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
gmaps.configure(api_key = map_key)

In [9]:
locations = city_df[["Lat", "Lon"]]
humidity = city_df["Humidity"]


fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 5


fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
weather_df = city_df[(city_df['Temp'] > 70) & (city_df['Temp'] < 80)]
weather_df = weather_df[weather_df['Wind Speed'] < 10]
weather_df = weather_df[weather_df['Cloudiness'] == 0]
vacation_df = weather_df
vacation_df

,Unnamed: 0,City ID,City Name,Country,Lat,Lon,Temp,Humidity,Cloudiness,Wind Speed
138,138,3533462,Acapulco de Juárez,MX,16.8634,-99.8901,78.87,81,0,4.70
306,306,286621,Salalah,OM,17.0151,54.0924,75.29,35,0,5.75
357,357,286245,Sur,OM,22.5667,59.5289,71.60,57,0,4.36
403,403,4026082,Lázaro Cárdenas,MX,17.9583,-102.2000,78.46,81,0,3.11
558,558,934649,Cap Malheureux,MU,-19.9842,57.6142,74.46,77,0,3.00


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
vacation_df['Hotel Name'] = ""
vacation_df

,Unnamed: 0,City ID,City Name,Country,Lat,Lon,Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
138,138,3533462,Acapulco de Juárez,MX,16.8634,-99.8901,78.87,81,0,4.70,
306,306,286621,Salalah,OM,17.0151,54.0924,75.29,35,0,5.75,
357,357,286245,Sur,OM,22.5667,59.5289,71.60,57,0,4.36,
403,403,4026082,Lázaro Cárdenas,MX,17.9583,-102.2000,78.46,81,0,3.11,
558,558,934649,Cap Malheureux,MU,-19.9842,57.6142,74.46,77,0,3.00,


In [12]:
for index, row in vacation_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": "map_key" ,
        }


        lat = row['Lat']
        lon = row['Lon']

        params['location'] = f"{lat}, {lon}"

        hotel_data = requests.get(base_url, params=params).json()
        
        vacation_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        vacation_df.loc[index, "Hotel Name"] = "NaN"
        
vacation_df     

,Unnamed: 0,City ID,City Name,Country,Lat,Lon,Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
138,138,3533462,Acapulco de Juárez,MX,16.8634,-99.8901,78.87,81,0,4.70,Hotel Emporio Acapulco
306,306,286621,Salalah,OM,17.0151,54.0924,75.29,35,0,5.75,OYO 115 Star Emirates Furnished Apartment
357,357,286245,Sur,OM,22.5667,59.5289,71.60,57,0,4.36,Sur Plaza Hotel
403,403,4026082,Lázaro Cárdenas,MX,17.9583,-102.2000,78.46,81,0,3.11,Baymont by Wyndham Lazaro Cardenas
558,558,934649,Cap Malheureux,MU,-19.9842,57.6142,74.46,77,0,3.00,Royal Palm Beachcomber Luxury


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]
locations = vacation_df[["Lat", "Lon"]]

In [14]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))